In [ ]:
# import pandas as pd
import numpy as np

In [40]:
df = pd.read_excel('quiqua.xlsx', sep = ";")

In [41]:
df.head()

,Renda,Estado,Escolaridade,Anos_Estudo,Estado_Civil
0,5948,MG,Superior,9,Solteiro
1,5870,MG,Superior,8,Solteiro
2,5710,GO,Superior,9,Casado
3,5660,RJ,Médio,7,Casado
4,5351,SP,Superior,8,Solteiro


# Frequencias

In [57]:
#Absoluta
pd.value_counts(df['Estado'])

SP    5
RJ    4
MG    4
TO    3
GO    3
RS    2
Name: Estado, dtype: int64

In [58]:
#Relativa
(pd.value_counts(df['Estado'])/len(df))

SP    0.238095
RJ    0.190476
MG    0.190476
TO    0.142857
GO    0.142857
RS    0.095238
Name: Estado, dtype: float64

In [59]:
pd.crosstab(df['Estado'],df['Escolaridade']).apply(lambda r: r/r.sum(), axis=1) #total por colunas

Escolaridade,Fundamental,Médio,Superior
Estado,,,
GO,0.666667,0.000000,0.333333
MG,0.500000,0.000000,0.500000
RJ,0.500000,0.500000,0.000000
RS,0.500000,0.000000,0.500000
SP,0.000000,0.800000,0.200000
TO,0.333333,0.333333,0.333333


In [60]:
pd.crosstab(df['Estado'],df['Escolaridade']).apply(lambda r: r/r.sum(), axis=0) #total por linhas

Escolaridade,Fundamental,Médio,Superior
Estado,,,
GO,0.250,0.000000,0.166667
MG,0.250,0.000000,0.333333
RJ,0.250,0.285714,0.000000
RS,0.125,0.000000,0.166667
SP,0.000,0.571429,0.166667
TO,0.125,0.142857,0.166667


# Medidas - Univariado

In [42]:
import statistics as st
print(df['Renda'].mean()) #calculo da média
print(st.median(df['Renda'])) #mediana
print(st.stdev(df['Renda'])) #desvio-padrao amostral
print(st.stdev(df['Renda'])/df['Renda'].mean()) #coeficiente de variacao - desvio padrao dividido pela media

3852.5238095238096
4454
1751.1335648387196
0.45454191886102013


In [43]:
df['Renda'].quantile([.1,.5,.75]) #quantis

0.10    1452.0
0.50    4454.0
0.75    5312.0
Name: Renda, dtype: float64

In [44]:
# Esse método retorna o valor de simetria de cada coluna do dataset.
# Um valor zero indica uma distribuição simétrica
# Um valor maior que zero ou menor indica uma distribuição assimétrica.
# Valores acima de zero podemos dizer que existe uma assimetria positiva 
# Valores abaixo de zero uma assimetria negativa.
# Isso quer dizer que valores muito acima de zero indicam que existem mais valores acima da média
# valores abaixo de zero significa que contém mais valores abaixo da média.
df['Renda'].skew()

-0.39913588796144006

# Medidas - Multivariado

In [45]:
#Correlação de spearman entre colunas quanti
df[['Renda','Anos_Estudo']].corr('spearman')

,Renda,Anos_Estudo
Renda,1.000000,0.927955
Anos_Estudo,0.927955,1.000000


In [13]:
from scipy.stats import chi2_contingency

In [46]:
#Calcular qui-quadrado (variaveis categoricas)
CATVAR = df.select_dtypes(include=['object']).columns.tolist() #cria uma lista das variaveis categoricas
chimap = pd.DataFrame(columns=['Var1','Var2','xsq','pvalue','vcramer']) #cria dataframe para inserir valores do chisquare
for x in CATVAR:
    for y in CATVAR:
        if x != y:
            chi = df.pivot_table('Renda',index= [x],columns= [y] ,aggfunc = 'count')
            chi=chi.replace(np.nan,0)
            xsq,pvalue,dof,expected=chi2_contingency(chi)
            a = min(len(df[x].unique()),len(df[y].unique()))
            b = len(df[x])
            vcramer = (xsq/(b*(a-1)))**(1/2)
            chimap = chimap.append({'Var1' : x, 'Var2' : y, 'xsq': xsq, 'pvalue': pvalue, 'vcramer':vcramer}, ignore_index=True)
chimap = chimap.sort_values(by = 'vcramer', ascending = False)
chimap

,Var1,Var2,xsq,pvalue,vcramer
1,Estado,Estado_Civil,18.725000,0.043898,0.667708
4,Estado_Civil,Estado,18.725000,0.043898,0.667708
0,Estado,Escolaridade,11.820833,0.297226,0.530517
2,Escolaridade,Estado,11.820833,0.297226,0.530517
3,Escolaridade,Estado_Civil,4.187500,0.381225,0.315757
5,Estado_Civil,Escolaridade,4.187500,0.381225,0.315757


In [47]:
CATVAR = df.select_dtypes(include=['object']).columns.tolist()
NUMVAR = df.select_dtypes(include=['float64','int64']).columns.tolist()

In [48]:
#Calculo do R2

In [49]:
r2 = pd.DataFrame(columns=['VarQuanti','VarQuali','var*n / n_total','R2'])
for z in NUMVAR:
    for x in CATVAR:
        dfr = df[x].value_counts().reset_index()
        f = 0
        a1 = 0
        for y in range(len(dfr)):
            a = dfr.iloc[y,1]
            a1 = a1 + a
            c = (df[df[x] == dfr.iloc[y,0]])
            e = (c[z].var())*a
            f = f + e
        f = f/a1
        r2_calc = 1 - (f/df[z].var())
        r2 = r2.append({'VarQuanti' : z, 'VarQuali' : x,'var*n / n_total' : f, 'R2' : r2_calc}, ignore_index=True)
r2 = r2.sort_values(by = 'R2', ascending = False)
r2

,VarQuanti,VarQuali,var*n / n_total,R2
1,Renda,Escolaridade,1.279860e+06,0.582627
4,Anos_Estudo,Escolaridade,3.138322e+00,0.497677
5,Anos_Estudo,Estado_Civil,6.480159e+00,-0.037221
2,Renda,Estado_Civil,3.307904e+06,-0.078734
3,Anos_Estudo,Estado,7.436508e+00,-0.190295
0,Renda,Estado,3.937980e+06,-0.284207


# Tabelas Cruzadas

In [50]:
df.pivot_table('Renda',index=["Estado","Estado_Civil"], aggfunc='mean',margins=True) 
#media da renda por estado e estado_civil

Renda
Estado Estado_Civil             
GO     Casado        3327.333333
MG     Casado        4950.000000
       Solteiro      4323.333333
RJ     Casado        5660.000000
       Solteiro      3642.000000
       Viuvo         2323.000000
RS     Casado        3401.000000
SP     Solteiro      3999.750000
       Viuvo         5119.000000
TO     Casado        4337.000000
       Viuvo         2738.500000
All                  3852.523810

In [55]:
df.pivot_table('Renda',index=["Estado"],columns=['Escolaridade'], aggfunc='count',margins=True) 
#quantas observacoes por estado e escolaridade, se colocar dropna = True, exclui os NaN

Escolaridade,Fundamental,Médio,Superior,All
Estado,,,,
GO,2.0,NaN,1.0,3
MG,2.0,NaN,2.0,4
RJ,2.0,2.0,NaN,4
RS,1.0,NaN,1.0,2
SP,NaN,4.0,1.0,5
TO,1.0,1.0,1.0,3
All,8.0,7.0,6.0,21


In [51]:
df.pivot_table('Renda',index=["Estado","Estado_Civil"], columns=['Escolaridade'] ,aggfunc='mean',margins=True)
#media da renda por estado, estado_civil e escolaridade

Escolaridade         Fundamental        Médio     Superior          All
Estado Estado_Civil                                                    
GO     Casado           2136.000          NaN  5710.000000  3327.333333
MG     Casado           4950.000          NaN          NaN  4950.000000
       Solteiro         1152.000          NaN  5909.000000  4323.333333
RJ     Casado                NaN  5660.000000          NaN  5660.000000
       Solteiro         2517.000  4767.000000          NaN  3642.000000
       Viuvo            2323.000          NaN          NaN  2323.000000
RS     Casado           1494.000          NaN  5308.000000  3401.000000
SP     Solteiro              NaN  3549.333333  5351.000000  3999.750000
       Viuvo                 NaN  5119.000000          NaN  5119.000000
TO     Casado                NaN          NaN  4337.000000  4337.000000
       Viuvo            1023.000  4454.000000          NaN  2738.500000
All                     2216.375  4378.285714  5420.666667  3852.523810

In [52]:
#transformando a tabela cruzada em df, usando reset.index()
df2 = (df.pivot_table('Renda',index=["Estado","Estado_Civil"], columns=['Escolaridade'] ,aggfunc='mean',margins=True)).reset_index()

In [53]:
df2

Escolaridade,Estado,Estado_Civil,Fundamental,Médio,Superior,All
0,GO,Casado,2136.000,NaN,5710.000000,3327.333333
1,MG,Casado,4950.000,NaN,NaN,4950.000000
2,MG,Solteiro,1152.000,NaN,5909.000000,4323.333333
3,RJ,Casado,NaN,5660.000000,NaN,5660.000000
4,RJ,Solteiro,2517.000,4767.000000,NaN,3642.000000
5,RJ,Viuvo,2323.000,NaN,NaN,2323.000000
6,RS,Casado,1494.000,NaN,5308.000000,3401.000000
7,SP,Solteiro,NaN,3549.333333,5351.000000,3999.750000
8,SP,Viuvo,NaN,5119.000000,NaN,5119.000000
9,TO,Casado,NaN,NaN,4337.000000,4337.000000
